In [23]:
import pandas as pd
import numpy as np

In [28]:
class MatchRating:
    original_rating_name = "Original"

    def __init__(self, player_file, match_file):
        self.players = pd.read_csv(player_file)
        self.players.set_index('ShortName', inplace=True)

        self.matches = pd.read_csv(match_file)

        self.__player_ratings = pd.DataFrame(columns=["Rating"], index=self.players.index)
        self.__player_ratings["Rating"] = np.random.normal(0, 1, self.__player_ratings.shape[0])
        self.__player_ratings

    def calculate_ratings(self, max_iterations):

        total_match_weights  = self.matches["K"].sum()

        for i in range(max_iterations):
            rating_table = self.__create_rating_table()
            
            match_score_accuracy = self.__calculate_match_score_accuracy(rating_table)        
            
            player_rating_accuracy = pow(match_score_accuracy.prod(), 1 / total_match_weights)
            accuracy_product = player_rating_accuracy.sort_values(ascending=False)

            best_accuracy = accuracy_product.index[0]
            print(f"Iteration {i + 1} with accuracy: {accuracy_product.iloc[0]}")

            if best_accuracy == MatchRating.original_rating_name:
                return self.__player_ratings.sort_values(by="Rating", ascending=False)
            
            player, rating_change = best_accuracy            
            self.__player_ratings.loc[player] += rating_change
            
        return self.__player_ratings.sort_values(by="Rating", ascending=False)

    def __calculate_match_score_accuracy(self, rating_table):
        match_score_accuracy = pd.DataFrame(columns=rating_table.columns, index=self.matches.index)

        for index, match in self.matches.iterrows():
            winner = match["Winner"]
            looser = match["Looser"]

            for column in rating_table.columns:
                winner_rating = rating_table.loc[winner][column]
                looser_rating = rating_table.loc[looser][column]

                accuracy = pow(winner_rating / (winner_rating + looser_rating), match["K"])

                match_score_accuracy.loc[index][column] = accuracy

        return match_score_accuracy

    def __create_rating_table(self):
        ratings = []
        columns = []

        original_rating = self.__player_ratings["Rating"]

        def add_rating(rating, name):
            ratings.append(rating.apply(lambda x: pow(10, (x / 40))))
            columns.append(name)

        def add_alternative_rating(player, rating_change):
            alternative_rating = original_rating.copy()
            alternative_rating.loc[player] += rating_change
                
            add_rating(alternative_rating, (player, rating_change))

        add_rating(original_rating, MatchRating.original_rating_name)

        for player in self.players.index:
            add_alternative_rating(player, 1)
            add_alternative_rating(player, -1)
            
        rating_table = pd.concat(ratings, axis=1)
        rating_table.columns = columns
        return rating_table

In [29]:
rating_system = MatchRating('Data/Players2.csv', 'Data/Matches2.csv')
rating_system.calculate_ratings(1000)

Iteration 1 with accuracy: 0.4974258248258957
Iteration 2 with accuracy: 0.4983069626874349
Iteration 3 with accuracy: 0.4991925277365539
Iteration 4 with accuracy: 0.500043242922176
Iteration 5 with accuracy: 0.500893913757173
Iteration 6 with accuracy: 0.5017526813328724
Iteration 7 with accuracy: 0.5025681893050665
Iteration 8 with accuracy: 0.5033898385473118
Iteration 9 with accuracy: 0.5042146396928265
Iteration 10 with accuracy: 0.5050055556063117
Iteration 11 with accuracy: 0.5057868214162372
Iteration 12 with accuracy: 0.5065771469385204
Iteration 13 with accuracy: 0.5073431799778224
Iteration 14 with accuracy: 0.5080837683888833
Iteration 15 with accuracy: 0.5088391940241076
Iteration 16 with accuracy: 0.5095801167930376
Iteration 17 with accuracy: 0.5103127740368726
Iteration 18 with accuracy: 0.5110197692207
Iteration 19 with accuracy: 0.5117410011167757
Iteration 20 with accuracy: 0.5124608326995709
Iteration 21 with accuracy: 0.5131716778300502
Iteration 22 with accuracy:

,Rating
ShortName,
JP,56.966577
MRF,51.962723
IEY,50.959849
RAI,46.611121
TNR,34.815993
MAD,27.692582
EMB,23.614964
VB,21.994200
SZP,21.053800
